[View in Colaboratory](https://colab.research.google.com/github/nunenuh/nnresnet/blob/master/collab_resnet.ipynb)

In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from keras import utils
from keras.models import Model
from keras.layers import *
from keras.datasets import cifar10

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

In [0]:
# load dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train[:20000]
y_train = y_train[:20000]

x_test = x_test[:5000]
y_test = y_test[:5000]

In [0]:
# normalisasi dataset
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

num_class = 10
y_train = utils.np_utils.to_categorical(y_train, num_class)
y_test = utils.np_utils.to_categorical(y_test, num_class)

In [0]:
# arsitektur
def residual_block(inputz, filterz):
    bc1 = Conv2D(filterz, kernel_size=(3,3), padding='same')(inputz)
    bbn1 = BatchNormalization()(bc1)
    bac1 = Activation('relu')(bbn1)
    dr1 = Dropout(0.25)(bac1)
    bc2 = Conv2D(filterz, kernel_size=(3,3), padding='same')(bac1)
    bbn2 = BatchNormalization()(bc2)
    add1 = Concatenate()([inputz, bbn2])
    bac2 = Activation('relu')(add1)
    dr1 = Dropout(0.25)(bac1)
    return bac2

# ishape = x_train[0].shape
ishape = (32, 32, 3)

inp = Input(shape=ishape)
# 32x32
c1 = Conv2D(filters=32, kernel_size=(7,7), strides=(2,2), padding='same')(inp)
# 16x16

p1 = MaxPooling2D(pool_size=(2, 2))(c1)
# 8x8
r1 = residual_block(inputz=p1, filterz=32)
r11 = residual_block(inputz=r1, filterz=32)
r12 = residual_block(inputz=r11, filterz=32)

# 8x8

p2 = MaxPooling2D(pool_size=(2, 2))(r12)
# 4x4
r2 = residual_block(inputz=p2, filterz=64)
r21 = residual_block(inputz=r2, filterz=64)
r22 = residual_block(inputz=r21, filterz=64)

# 4x4

p4 = AveragePooling2D(pool_size=(2,2))(r22)
fl1 = Flatten()(p4)
fc = Dense(num_class, activation='softmax')(fl1)
output = fc

model = Model(inp, output)
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()
# residual block


In [0]:
!wget https://pbs.twimg.com/media/Cdw8VlSWAAAB9FF.jpg

In [0]:
!ls -al

In [0]:
model.fit(
    x_train, y_train,
    validation_data=(x_test,y_test),
    epochs=10, batch_size=64
)